In [ ]:
import csv
import re
from lxml import etree
from svgpathtools import svg2paths
import pandas as pd
import numpy as np
import os

In [ ]:
def readblankmap(filename = "states_svg/Blank map subdivisions 2019 Alabama.svg"):
    with open(filename, "r") as f:
        svg = f.read()
    return(svg)


In [ ]:
def editmap(svg, bounds):
    svg = svg.replace('style="stroke:#000000"', 'style="stroke:#ffffff"')
    svg = svg.replace('style="fill:none;stroke:#000000;', 
                      'style="fill:none;stroke:#ffffff;')
    xmin, xmax, ymin, ymax = bounds
#     svg = svg.replace('<g id="counties" transform="translate(0,0.10698)"',
#                      '<g id="counties" transform="translate(' + str(-xmin) + ',' + str(-ymin) + ')"')
    svg = svg.replace('fill="#d0d0d0" height="100%" stroke="#ffffff" stroke-width="1" version="1.1" width="100%" xmlns="http://www.w3.org/2000/svg"',
                      'fill="#d0d0d0" height="' + str(ymax - ymin) + '" stroke="#ffffff" stroke-width="1" version="1.1" width="' + str(xmax - xmin) + '" xmlns="http://www.w3.org/2000/svg"')
    svg = svg.replace('><defs /><g ', '><defs />\n\t<g ')
    svg = svg.replace('><defs />', '>\n\t<defs />')
    svg = svg.replace('/><path ', '/>\n\t\t<path ')
    svg = svg.replace('/></g></svg>', '/>\n\t</g>\n</svg>')
    return(svg)

In [ ]:
def writesvg(svg, filename = "output/test.svg"):
    with open(filename, "w") as f:
        f.write(svg)
    print("written: " + filename)

In [ ]:
def findminmax (filename):
    paths, attributes = svg2paths(filename)
    xmin, xmax, ymin, ymax = paths[0].bbox()
    for path in paths:
        xmini, xmaxi, ymini, ymaxi = path.bbox()
        xmin = min(xmin, xmini)
        xmax = max(xmax, xmaxi)
        ymin = min(ymin, ymini)
        ymax = max(ymax, ymaxi)
    return([xmin, xmax, ymin, ymax])

In [ ]:
file_list = []
script_dir = os.path.dirname(os.path.realpath('maps'))
folderPath = "states_svg_albers"
for base_path, dir_names, file_names in os.walk(script_dir + '/' + folderPath):
    for file_name in file_names:
        if file_name.endswith(".svg") or \
        file_name.endswith(".png") or \
        file_name.endswith(".jpg") or \
        file_name.endswith(".jpeg") or \
        file_name.endswith(".gif"):
            file_path = os.path.join(base_path, file_name)
            rel_path = os.path.relpath(file_path, script_dir)
            file_list.append([rel_path, file_name])

In [ ]:
file_list[0]

In [ ]:
census = pd.read_csv('data/census.csv')
states = census.groupby('STATE').agg({'STNAME': 'first'})
states_df = pd.DataFrame(states)
for file in file_list:
#     st_index = str(state_id).zfill(2)
#     st_name = states_df.loc[state_id]["STNAME"]
#     filtermap(filename = "Usa_counties_large.svg", state_id = st_index)
    bounds_state = findminmax(file[0])
    print(bounds_state)
    svg_state = readblankmap(filename = file[0])
    edited_svg_state = editmap(svg_state, bounds = bounds_state)
    writesvg(svg = edited_svg_state, filename = "output_states_svg_albers/Blank map subdivisions 2019 Albers " + file[1][3:])